<a href="https://www.kaggle.com/code/notsau/cnn-with-parasite-data?scriptVersionId=161440885" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Data

In [1]:
#Veri çok büyük 5000 tanesini alıp yükledik

In [2]:
import os
import random
import shutil

# Veri seti yolu
data_dir = '/kaggle/input/cell-images-for-detecting-malaria/cell_images'
hedef_dir = 'veri'

# Rastgele fotoğrafların sayısı
secilen_sayi = 5000

# Parasitized fotoğrafların dizini
parasitized_dir = os.path.join(data_dir, 'Parasitized')

# Uninfected fotoğrafların dizini
uninfected_dir = os.path.join(data_dir, 'Uninfected')

# Hedef dizinler
hedef_parasitized_dir = os.path.join(hedef_dir, 'parazitli')
hedef_uninfected_dir = os.path.join(hedef_dir, 'parazitsiz')

# Hedef dizinleri oluşturma
os.makedirs(hedef_parasitized_dir, exist_ok=True)
os.makedirs(hedef_uninfected_dir, exist_ok=True)

# Parasitized fotoğrafları rastgele seçme
parasitized_fotograflar = os.listdir(parasitized_dir)
secilen_parasitizedler = random.sample(parasitized_fotograflar, secilen_sayi)

# Uninfected fotoğrafları rastgele seçme
uninfected_fotograflar = os.listdir(uninfected_dir)
secilen_uninfectedler = random.sample(uninfected_fotograflar, secilen_sayi)

# Parasitized fotoğrafları hedef dizine kopyalama
for parasitized in secilen_parasitizedler:
    kaynak = os.path.join(parasitized_dir, parasitized)
    hedef = os.path.join(hedef_parasitized_dir, parasitized)
    shutil.copyfile(kaynak, hedef)

# Uninfected fotoğrafları hedef dizine kopyalama
for uninfected in secilen_uninfectedler:
    kaynak = os.path.join(uninfected_dir, uninfected)
    hedef = os.path.join(hedef_uninfected_dir, uninfected)
    shutil.copyfile(kaynak, hedef)

In [3]:
#Test/Eğitim Dosyaları Oluşturma

In [4]:
import os
import random
import shutil

# Veri seti dizini
base_dir = 'veri'
train_dir = os.path.join(base_dir, 'egitim')
test_dir = os.path.join(base_dir, 'test')

# Eğitim ve test dizinlerini oluşturma
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Sınıf isimleri
class_names = ['parazitli', 'parazitsiz']

# Eğitim ve test veri setlerini oluşturma
for class_name in class_names:
    class_dir = os.path.join(base_dir, class_name)
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)
    
    # Sınıfa ait tüm fotoğrafları elde etme
    all_files = os.listdir(class_dir)
    random.shuffle(all_files)
    
    # Eğitim ve test veri setlerine fotoğrafları kopyalama
    split_index = int(0.8 * len(all_files))  # %80 eğitim, %20 test
    train_files = all_files[:split_index]
    test_files = all_files[split_index:]
    
    for train_file in train_files:
        src = os.path.join(class_dir, train_file)
        dst = os.path.join(train_class_dir, train_file)
        shutil.copyfile(src, dst)
        
    for test_file in test_files:
        src = os.path.join(class_dir, test_file)
        dst = os.path.join(test_class_dir, test_file)
        shutil.copyfile(src, dst)

# Modelling

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Veri seti yolu
base_dir = 'veri'
train_dir = os.path.join(base_dir, 'egitim')
test_dir = os.path.join(base_dir, 'test')

# Veri ön işleme
image_size = (150, 150)
batch_size = 64

# Eğitim veri seti için veri artırma ve normalizasyon
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Test veri seti için sadece normalizasyon
test_datagen = ImageDataGenerator(rescale=1./255)

# Eğitim ve test veri setlerini yükleyin ve ön işleyin
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Model oluşturma
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Model derleme
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model eğitimi
epochs = 1
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)




Found 19177 images belonging to 2 classes.

Found 7936 images belonging to 2 classes.
















299/299 [==============================] - 394s 1s/step - loss: 0.4334 - accuracy: 0.7995 - val_loss: 0.2074 - val_accuracy: 0.9286
